In [0]:
-- Insert generated calendar data into the dim_calendar table
INSERT INTO workspace.insulet_uk_curated.dim_calendar
-- Generate a sequence of dates from 2010-01-01 to 2030-12-31
WITH date_sequence AS (
  SELECT explode(
    sequence(
      DATE('2010-01-01'),
      DATE('2030-12-31'),
      INTERVAL 1 DAY
    )
  ) AS full_date
),
calendar_dates AS (
  SELECT
    -- Format date as YYYYMMDD for date_id
    REPLACE(CAST(full_date AS STRING), '-', '') AS date_id,
    full_date,
    -- Extract day of month
    DAY(full_date) AS day_of_month,
    -- Extract day of week name
    DATE_FORMAT(full_date, 'EEEE') AS day_of_week,
    -- Extract calendar month
    MONTH(full_date) AS calendar_month,
    -- Extract calendar quarter
    QUARTER(full_date) AS calendar_quarter,
    -- Extract calendar year
    YEAR(full_date) AS calendar_year,
    -- Calculate fiscal year (starts 6th April)
    CASE
      WHEN MONTH(full_date) > 4 OR (MONTH(full_date) = 4 AND DAY(full_date) >= 6)
        THEN YEAR(full_date)
      ELSE YEAR(full_date) - 1
    END AS fiscal_year,
    -- Calculate fiscal month: April (from 6th) is month 1, May is 2, ..., March is 12
    CASE
      WHEN MONTH(full_date) > 4 OR (MONTH(full_date) = 4 AND DAY(full_date) >= 6)
        THEN MONTH(full_date) - 3
      ELSE MONTH(full_date) + 9
    END AS fiscal_month,
    -- Calculate fiscal quarter: Q1 = Apr-Jun, Q2 = Jul-Sep, Q3 = Oct-Dec, Q4 = Jan-Mar
    CASE
      WHEN (MONTH(full_date) > 4 OR (MONTH(full_date) = 4 AND DAY(full_date) >= 6))
        THEN CEIL((MONTH(full_date) - 3) / 3.0)
      ELSE CEIL((MONTH(full_date) + 9) / 3.0)
    END AS fiscal_quarter,
    -- Generate a hash key for the calendar row
    sha2(concat_ws('^', CAST(full_date AS STRING)), 256) AS calendar_hash_key,
    -- Record load timestamp
    current_timestamp() AS load_date,
    -- Source system identifier
    'calendar_generator' AS source_system
  FROM date_sequence
)
SELECT
  calendar_hash_key,
  date_id,
  full_date,
  day_of_month,
  day_of_week,
  calendar_month,
  calendar_quarter,
  calendar_year,
  fiscal_month,
  fiscal_quarter,
  fiscal_year,
  load_date,
  source_system
FROM calendar_dates